# Lab 1.3.2: Optimizer Implementation - SOLUTIONS

This notebook contains complete solutions to all exercises in the optimizer implementation lab.

---

> **📝 Note:** This solution notebook is designed to be self-contained and includes all necessary
> helper functions. However, if you prefer to use your own implementations from the main notebook,
> ensure those cells have been run first.
>
> You can also import production-ready implementations from the scripts:
> ```python
> from scripts.math_utils import sigmoid, relu, Adam, SGD
> from scripts.visualization_utils import plot_loss_landscape, plot_training_curve
> ```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

np.random.seed(42)

print("Solutions Notebook - Optimizer Implementation")
print("=" * 50)

In [ ]:
# Helper functions from the main notebook

def rosenbrock(x, y, a=1, b=100):
    """Rosenbrock function"""
    return (a - x)**2 + b * (y - x**2)**2

def rosenbrock_gradient(x, y, a=1, b=100):
    """Gradient of Rosenbrock function"""
    df_dx = -2*(a - x) - 4*b*x*(y - x**2)
    df_dy = 2*b*(y - x**2)
    return np.array([df_dx, df_dy])

def optimize(optimizer, start_point, gradient_fn, n_steps=1000):
    """Run optimization and track history"""
    optimizer.reset()
    params = start_point.copy()
    history = {'params': [params.copy()], 'loss': [rosenbrock(params[0], params[1])]}
    
    for _ in range(n_steps):
        grads = gradient_fn(params[0], params[1])
        params = optimizer.step(params, grads)
        loss = rosenbrock(params[0], params[1])
        history['params'].append(params.copy())
        history['loss'].append(loss)
        if loss < 1e-10:
            break
    
    return history

start_point = np.array([-1.0, 1.0])

---

## Solution 1: RMSprop Optimizer

**Exercise:** Implement RMSprop optimizer.

RMSprop maintains a moving average of squared gradients and uses it to normalize the gradient:

$$v_t = \beta \cdot v_{t-1} + (1-\beta) \cdot g_t^2$$
$$\theta_{t+1} = \theta_t - \frac{\eta}{\sqrt{v_t} + \epsilon} \cdot g_t$$

In [ ]:
# SOLUTION: RMSprop optimizer

class RMSprop:
    """
    RMSprop optimizer.
    
    RMSprop adapts the learning rate for each parameter based on the
    magnitude of recent gradients. Parameters with large gradients get
    smaller updates, and vice versa.
    
    Update rules:
        v = β × v + (1-β) × g²     (exponential moving average of squared gradients)
        θ = θ - lr × g / (√v + ε)  (normalized update)
    
    Args:
        lr: Learning rate (default 0.01)
        beta: Decay rate for moving average (default 0.9)
        epsilon: Small constant for numerical stability (default 1e-8)
    
    Example:
        >>> optimizer = RMSprop(lr=0.01, beta=0.9)
        >>> params = optimizer.step(params, grads)
    """
    
    def __init__(self, lr=0.01, beta=0.9, epsilon=1e-8):
        self.lr = lr
        self.beta = beta
        self.epsilon = epsilon
        self.v = None  # Moving average of squared gradients
        self.name = f"RMSprop (lr={lr}, β={beta})"
    
    def step(self, params, grads):
        """
        Perform one RMSprop optimization step.
        
        Args:
            params: Current parameters (numpy array)
            grads: Gradients at current parameters
            
        Returns:
            Updated parameters
        """
        # Initialize v on first call
        if self.v is None:
            self.v = np.zeros_like(params)
        
        # Update moving average of squared gradients
        # v = β*v + (1-β)*g²
        self.v = self.beta * self.v + (1 - self.beta) * (grads ** 2)
        
        # Update parameters with normalized gradient
        # θ = θ - lr * g / (√v + ε)
        params = params - self.lr * grads / (np.sqrt(self.v) + self.epsilon)
        
        return params
    
    def reset(self):
        """Reset optimizer state"""
        self.v = None

# Test RMSprop
print("Testing RMSprop Optimizer")
print("=" * 50)

rmsprop = RMSprop(lr=0.01, beta=0.9)
history_rmsprop = optimize(rmsprop, start_point, rosenbrock_gradient, n_steps=5000)

final_params = history_rmsprop['params'][-1]
final_loss = history_rmsprop['loss'][-1]

print(f"Start: ({start_point[0]}, {start_point[1]})")
print(f"Goal:  (1.0, 1.0)")
print(f"Final: ({final_params[0]:.6f}, {final_params[1]:.6f})")
print(f"Final loss: {final_loss:.6e}")
print(f"Steps: {len(history_rmsprop['loss']) - 1}")

In [ ]:
# Visualize RMSprop performance

plt.figure(figsize=(10, 5))
plt.semilogy(history_rmsprop['loss'], 'b-', linewidth=2)
plt.xlabel('Step', fontsize=12)
plt.ylabel('Loss (log scale)', fontsize=12)
plt.title('RMSprop Convergence on Rosenbrock Function', fontsize=14)
plt.grid(True, alpha=0.3)
plt.axhline(y=1e-6, color='r', linestyle='--', label='Target: 1e-6')
plt.legend()
plt.tight_layout()
plt.show()

### Key Differences: RMSprop vs Adam

| Feature | RMSprop | Adam |
|---------|---------|------|
| Momentum | ❌ No | ✅ Yes (first moment) |
| Adaptive LR | ✅ Yes (second moment) | ✅ Yes (second moment) |
| Bias correction | ❌ No | ✅ Yes |
| Typical use | RNNs | General purpose |

---

## Solution 2: AdamW Optimizer

**Exercise:** Implement AdamW (Adam with decoupled weight decay).

The key insight is that L2 regularization and weight decay are NOT the same with adaptive optimizers:
- L2 regularization adds `λ*w` to the gradient
- Weight decay directly subtracts `λ*w` from the weights

AdamW decouples weight decay from the gradient-based update.

In [ ]:
# SOLUTION: AdamW optimizer

class AdamW:
    """
    AdamW optimizer - Adam with decoupled weight decay.
    
    In standard Adam with L2 regularization, the regularization term
    is scaled by the adaptive learning rate, which can be problematic.
    
    AdamW fixes this by applying weight decay directly to the weights,
    separately from the gradient-based update.
    
    Update rules:
        m = β1 × m + (1-β1) × g           (momentum)
        v = β2 × v + (1-β2) × g²          (variance)
        m_hat = m / (1 - β1^t)            (bias correction)
        v_hat = v / (1 - β2^t)            (bias correction)
        θ = θ - lr × m_hat / (√v_hat + ε) (Adam update)
        θ = θ - lr × λ × θ                (weight decay - DECOUPLED!)
    
    Args:
        lr: Learning rate (default 0.001)
        beta1: Momentum coefficient (default 0.9)
        beta2: Variance coefficient (default 0.999)
        epsilon: Numerical stability (default 1e-8)
        weight_decay: Weight decay coefficient (default 0.01)
    
    Example:
        >>> optimizer = AdamW(lr=0.001, weight_decay=0.01)
        >>> params = optimizer.step(params, grads)
    """
    
    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8, weight_decay=0.01):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.weight_decay = weight_decay
        
        self.m = None  # First moment
        self.v = None  # Second moment
        self.t = 0     # Timestep
        
        self.name = f"AdamW (lr={lr}, wd={weight_decay})"
    
    def step(self, params, grads):
        """
        Perform one AdamW optimization step.
        
        Args:
            params: Current parameters
            grads: Gradients at current parameters
            
        Returns:
            Updated parameters
        """
        # Initialize on first call
        if self.m is None:
            self.m = np.zeros_like(params)
            self.v = np.zeros_like(params)
        
        self.t += 1
        
        # Update biased first moment (momentum)
        self.m = self.beta1 * self.m + (1 - self.beta1) * grads
        
        # Update biased second moment (variance)
        self.v = self.beta2 * self.v + (1 - self.beta2) * (grads ** 2)
        
        # Bias correction
        m_hat = self.m / (1 - self.beta1 ** self.t)
        v_hat = self.v / (1 - self.beta2 ** self.t)
        
        # Adam update
        params = params - self.lr * m_hat / (np.sqrt(v_hat) + self.epsilon)
        
        # DECOUPLED weight decay (this is the key difference from Adam+L2!)
        params = params - self.lr * self.weight_decay * params
        
        return params
    
    def reset(self):
        """Reset optimizer state"""
        self.m = None
        self.v = None
        self.t = 0

# Test AdamW
print("Testing AdamW Optimizer")
print("=" * 50)

adamw = AdamW(lr=0.1, weight_decay=0.01)
history_adamw = optimize(adamw, start_point, rosenbrock_gradient, n_steps=5000)

final_params = history_adamw['params'][-1]
final_loss = history_adamw['loss'][-1]

print(f"Start: ({start_point[0]}, {start_point[1]})")
print(f"Goal:  (1.0, 1.0)")
print(f"Final: ({final_params[0]:.6f}, {final_params[1]:.6f})")
print(f"Final loss: {final_loss:.6e}")

In [ ]:
# Compare Adam vs AdamW

class Adam:
    """Standard Adam optimizer"""
    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = None
        self.v = None
        self.t = 0
        self.name = f"Adam (lr={lr})"
    
    def step(self, params, grads):
        if self.m is None:
            self.m = np.zeros_like(params)
            self.v = np.zeros_like(params)
        self.t += 1
        self.m = self.beta1 * self.m + (1 - self.beta1) * grads
        self.v = self.beta2 * self.v + (1 - self.beta2) * (grads ** 2)
        m_hat = self.m / (1 - self.beta1 ** self.t)
        v_hat = self.v / (1 - self.beta2 ** self.t)
        return params - self.lr * m_hat / (np.sqrt(v_hat) + self.epsilon)
    
    def reset(self):
        self.m = None
        self.v = None
        self.t = 0

# Run both
adam = Adam(lr=0.1)
adamw = AdamW(lr=0.1, weight_decay=0.01)

history_adam = optimize(adam, start_point, rosenbrock_gradient, 5000)
history_adamw = optimize(adamw, start_point, rosenbrock_gradient, 5000)

# Plot comparison
plt.figure(figsize=(10, 5))
plt.semilogy(history_adam['loss'], 'b-', linewidth=2, label='Adam', alpha=0.8)
plt.semilogy(history_adamw['loss'], 'r--', linewidth=2, label='AdamW (wd=0.01)', alpha=0.8)
plt.xlabel('Step', fontsize=12)
plt.ylabel('Loss (log scale)', fontsize=12)
plt.title('Adam vs AdamW Comparison', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nAdam final loss:  {history_adam['loss'][-1]:.6e}")
print(f"AdamW final loss: {history_adamw['loss'][-1]:.6e}")

---

## Solution 3: Nesterov Accelerated Gradient (NAG)

**Challenge:** Implement Nesterov momentum.

The key insight of Nesterov momentum is to compute the gradient at the "lookahead" position:

$$v_t = \beta \cdot v_{t-1} + \nabla L(\theta_t - \beta \cdot v_{t-1})$$
$$\theta_{t+1} = \theta_t - \eta \cdot v_t$$

This "looks ahead" to where momentum would take us, then computes the gradient there.

In [ ]:
# SOLUTION: Nesterov Accelerated Gradient

class NesterovMomentum:
    """
    Nesterov Accelerated Gradient (NAG) optimizer.
    
    Unlike standard momentum which computes the gradient at the current
    position, Nesterov computes the gradient at the "lookahead" position
    (where momentum would take us).
    
    This often leads to faster convergence because we're evaluating the
    gradient at a more informed position.
    
    Update rules:
        lookahead = θ - β × v           (where momentum would take us)
        g = ∇L(lookahead)               (gradient at lookahead)
        v = β × v + g                   (update velocity with lookahead gradient)
        θ = θ - lr × v                  (update parameters)
    
    Note: This requires access to the gradient function, not just the gradient.
    
    Args:
        lr: Learning rate
        momentum: Momentum coefficient (typically 0.9)
    """
    
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.velocity = None
        self.name = f"Nesterov (lr={lr}, β={momentum})"
    
    def step(self, params, grads, gradient_fn=None):
        """
        Perform one Nesterov momentum step.
        
        Args:
            params: Current parameters
            grads: Gradients at current parameters (used if gradient_fn is None)
            gradient_fn: Optional function to compute gradients at any point
            
        Returns:
            Updated parameters
        """
        if self.velocity is None:
            self.velocity = np.zeros_like(params)
        
        if gradient_fn is not None:
            # True Nesterov: compute gradient at lookahead position
            lookahead = params - self.momentum * self.velocity
            grads = gradient_fn(lookahead[0], lookahead[1])
        
        # Update velocity with (lookahead) gradient
        self.velocity = self.momentum * self.velocity + grads
        
        # Update parameters
        return params - self.lr * self.velocity
    
    def reset(self):
        """Reset velocity"""
        self.velocity = None


def optimize_nesterov(optimizer, start_point, gradient_fn, n_steps=1000):
    """Special optimization loop for Nesterov (needs gradient_fn)"""
    optimizer.reset()
    params = start_point.copy()
    history = {'params': [params.copy()], 'loss': [rosenbrock(params[0], params[1])]}
    
    for _ in range(n_steps):
        # Pass gradient function for true Nesterov
        params = optimizer.step(params, None, gradient_fn=gradient_fn)
        loss = rosenbrock(params[0], params[1])
        history['params'].append(params.copy())
        history['loss'].append(loss)
        if loss < 1e-10:
            break
    
    return history

# Test Nesterov
print("Testing Nesterov Accelerated Gradient")
print("=" * 50)

nesterov = NesterovMomentum(lr=0.001, momentum=0.9)
history_nesterov = optimize_nesterov(nesterov, start_point, rosenbrock_gradient, 5000)

final_params = history_nesterov['params'][-1]
final_loss = history_nesterov['loss'][-1]

print(f"Final: ({final_params[0]:.6f}, {final_params[1]:.6f})")
print(f"Final loss: {final_loss:.6e}")

In [ ]:
# Compare standard momentum vs Nesterov

class SGDMomentum:
    """Standard SGD with momentum"""
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.velocity = None
        self.name = f"Momentum (lr={lr}, β={momentum})"
    
    def step(self, params, grads):
        if self.velocity is None:
            self.velocity = np.zeros_like(params)
        self.velocity = self.momentum * self.velocity + grads
        return params - self.lr * self.velocity
    
    def reset(self):
        self.velocity = None

# Run both
momentum_opt = SGDMomentum(lr=0.001, momentum=0.9)
nesterov_opt = NesterovMomentum(lr=0.001, momentum=0.9)

history_momentum = optimize(momentum_opt, start_point, rosenbrock_gradient, 5000)
history_nesterov = optimize_nesterov(nesterov_opt, start_point, rosenbrock_gradient, 5000)

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss curves
axes[0].semilogy(history_momentum['loss'], 'b-', linewidth=2, label='Standard Momentum', alpha=0.8)
axes[0].semilogy(history_nesterov['loss'], 'r--', linewidth=2, label='Nesterov Momentum', alpha=0.8)
axes[0].set_xlabel('Step', fontsize=12)
axes[0].set_ylabel('Loss (log scale)', fontsize=12)
axes[0].set_title('Momentum vs Nesterov: Convergence', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Trajectories
x_range = np.linspace(-1.5, 1.5, 100)
y_range = np.linspace(-0.5, 2, 100)
X, Y = np.meshgrid(x_range, y_range)
Z = rosenbrock(X, Y)

levels = np.logspace(-1, 3, 30)
axes[1].contour(X, Y, Z, levels=levels, cmap='Greys', alpha=0.5)

path_m = np.array(history_momentum['params'])
path_n = np.array(history_nesterov['params'])

step = max(1, len(path_m) // 100)
axes[1].plot(path_m[::step, 0], path_m[::step, 1], 'b-', linewidth=1.5, 
            label='Standard Momentum', alpha=0.7)
axes[1].plot(path_n[::step, 0], path_n[::step, 1], 'r--', linewidth=1.5,
            label='Nesterov Momentum', alpha=0.7)
axes[1].scatter([1], [1], color='gold', s=200, marker='*', zorder=5, label='Optimum')
axes[1].set_xlabel('x', fontsize=12)
axes[1].set_ylabel('y', fontsize=12)
axes[1].set_title('Momentum vs Nesterov: Trajectories', fontsize=14)
axes[1].legend(loc='upper left')

plt.tight_layout()
plt.show()

print(f"\nStandard Momentum final loss: {history_momentum['loss'][-1]:.6e}")
print(f"Nesterov Momentum final loss: {history_nesterov['loss'][-1]:.6e}")

### Why Nesterov Works Better

**Standard momentum:** "Where am I now? Let me add momentum and go."

**Nesterov momentum:** "If I keep going with my momentum, where will I be? Let me compute the gradient THERE."

This "lookahead" prevents overshooting by giving an early warning when the gradient changes direction.

---

## Bonus: All Optimizers Comparison

In [ ]:
# Compare all implemented optimizers

class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
        self.name = f"SGD (lr={lr})"
    def step(self, params, grads):
        return params - self.lr * grads
    def reset(self):
        pass

# Create all optimizers
optimizers = [
    SGD(lr=0.001),
    SGDMomentum(lr=0.001, momentum=0.9),
    RMSprop(lr=0.01, beta=0.9),
    Adam(lr=0.1),
    AdamW(lr=0.1, weight_decay=0.01),
]

# Run all
histories = {}
for opt in optimizers:
    histories[opt.name] = optimize(opt, start_point, rosenbrock_gradient, 5000)

# Add Nesterov separately (needs special loop)
nesterov = NesterovMomentum(lr=0.001, momentum=0.9)
histories[nesterov.name] = optimize_nesterov(nesterov, start_point, rosenbrock_gradient, 5000)

# Plot all
plt.figure(figsize=(12, 6))
colors = plt.cm.tab10(np.linspace(0, 1, len(histories)))

for (name, hist), color in zip(histories.items(), colors):
    plt.semilogy(hist['loss'], linewidth=2, label=name, color=color, alpha=0.8)

plt.xlabel('Step', fontsize=12)
plt.ylabel('Loss (log scale)', fontsize=12)
plt.title('All Optimizers Comparison on Rosenbrock Function', fontsize=14)
plt.legend(loc='upper right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Print final losses
print("\nFinal Losses:")
print("-" * 40)
for name, hist in sorted(histories.items(), key=lambda x: x[1]['loss'][-1]):
    print(f"{name:<35} {hist['loss'][-1]:.6e}")

---

## Key Takeaways

1. **RMSprop:** Adapts learning rate per-parameter using gradient magnitude
2. **AdamW:** Decouples weight decay from gradient scaling (better for transformers)
3. **Nesterov:** Computes gradient at "lookahead" position for faster convergence
4. **Adam/AdamW** are usually the best default choices for deep learning
5. **RMSprop** can be good for RNNs where Adam sometimes struggles